<a href="https://colab.research.google.com/github/Swayamprakashpatel/Sol_ME/blob/main/Sol_ME_Weblink.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pubchempy
!pip install gradio
!pip install gdown

import os
import tensorflow as tf
import pandas as pd
import numpy as np
import io
from tensorflow import keras
import pubchempy as pcp
import gradio as gr
import gdown
import time

# --- Setup for Model Download and Loading (Runs once at startup) ---

# This section downloads and loads your model so it's ready for predictions.
# The `if` check ensures the file isn't downloaded again if it already exists.
file_id = '19vWbg_CKrTmgTupKjVsNerZCQGo-gi0n'
destination = 'Sol_ME.hdf5'

# Define the model path and Google Drive URL
model_path = os.path.join(os.getcwd(), destination)
drive_url = f'https://drive.google.com/uc?id={file_id}'

# Check if the model file exists, if not, download it
if not os.path.exists(model_path):
    print("Downloading model...")
    gdown.download(drive_url, model_path, quiet=False)
    print("Download complete.")
else:
    print("Model file already exists. Skipping download.")

# Load the model once
try:
    model = keras.models.load_model(model_path, compile=False)
    print("Model loaded successfully.")
except Exception as e:
    print(f"Error loading model: {e}")
    model = None

# --- Main Prediction Function for Gradio ---
# This function takes the inputs from the web form and performs the prediction.
def predict_solubility(drug_cid, excipient_cid, logp_drug, logp_excipient, molwt_drug, molwt_excipient):
    """
    Predicts drug solubility based on chemical properties and fingerprints.

    Args:
        drug_cid (int): PubChem CID of the drug.
        excipient_cid (int): PubChem CID of the excipient.
        logp_drug (float): LogP value of the drug.
        logp_excipient (float): LogP value of the excipient.
        molwt_drug (float): Molecular weight of the drug.
        molwt_excipient (float): Molecular weight of the excipient.

    Returns:
        str: The predicted solubility in mg/ml or an error message.
    """
    # Check if the model was loaded successfully
    if model is None:
        return "Error: The prediction model could not be loaded."

    try:
        # Prepare the inputs as per your original code
        logp_D = logp_drug / 100
        logp_C = logp_excipient / 100

        molwt_D_scaled = molwt_drug / 1000
        molwt_E_scaled = molwt_excipient / 1000

        # Fetch fingerprints from PubChem
        print("Fetching fingerprints from PubChem...")
        cid_d_compound = pcp.Compound.from_cid(drug_cid)
        cid_e_compound = pcp.Compound.from_cid(excipient_cid)

        fpd = cid_d_compound.cactvs_fingerprint
        fpe = cid_e_compound.cactvs_fingerprint

        # Concatenate all features into a single list
        features = [molwt_D_scaled, molwt_E_scaled, logp_D, logp_C]
        features.extend(list(fpd))
        features.extend(list(fpe))

        # Convert to a NumPy array for prediction
        input_array = np.array(features).reshape(1, -1).astype(float)

        # Make prediction
        prediction = model.predict(input_array)

        # Format the output string
        result_text = f"Predicted Solubility: {prediction[0][0]:.4f} mg/ml"
        return result_text

    except pcp.NotFoundError:
        return "Error: One or both of the PubChem CIDs were not found. Please check and try again."
    except Exception as e:
        return f"An unexpected error occurred: {e}"

# --- Gradio Interface Setup ---

# Define the input components for the Gradio app
inputs = [
    gr.Number(label="Drug PubChem CID", value=24872560),
    gr.Number(label="Excipient PubChem CID", value=3314),
    gr.Number(label="Drug LogP", value=11.3),
    gr.Number(label="Excipient LogP", value=2.0),
    gr.Number(label="Drug Molecular Weight", value=11.3),
    gr.Number(label="Excipient Molecular Weight", value=2.0),
]

# Define the output component
output = gr.Label(label="Prediction Result")

# Create the Gradio interface
iface = gr.Interface(
    fn=predict_solubility,
    inputs=inputs,
    outputs=output,
    title="Drug Solubility Prediction",
    description="Enter the PubChem CIDs and chemical properties of a drug and excipient to predict the solubility in mg/ml."
)

# Launch the app
iface.launch()